In [22]:
import torch
# from LocalStructure.ipynb import LCS
from tqdm import tqdm
import numpy as np
from skmultilearn.dataset import load_dataset
from skmultilearn.dataset import available_data_sets
from scipy.io import loadmat
from sklearn.cluster import KMeans
from skmultilearn.adapt import MLkNN
from sklearn.neighbors import kneighbors_graph
import sklearn
from numpy.matlib import repmat
import matplotlib.pyplot as plt
import pandas as pd
# Metrics
from sklearn.metrics import coverage_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import label_ranking_loss
from sklearn.metrics import average_precision_score
from sklearn.metrics import hamming_loss
import scipy.sparse as sp
from pathlib import Path
import os
import warnings
warnings.filterwarnings('ignore')

In [23]:
## GPU or CPU
GPU = False
if GPU:
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    print("num GPUs", torch.cuda.device_count())
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
    print("CPU")

CPU



\begin{align*}
    \normalsize{Multilabel \hspace{0.1cm} Feature Selection \hspace{0.1cm} With \hspace{0.1cm} Constrained \hspace{0.1cm} Latent \hspace{0.1cm} Structure \hspace{0.1cm} Shared \hspace{0.1cm}Term}
\end{align*}
\begin{align*}
    \normalsize{(2021)}
\end{align*}
$$
\begin{align*}
    \text{Wanfu  Gao, Yonghao Li, Liang  Hu }
\end{align*}
$$

$\normalsize{Updates \hspace{0.1cm} Formulas:}$

<div style="background-color: #EEEFFF; padding: 10px;">

\begin{align*}
    \Large{V \odot V \frac{XQ + \alpha YM^\top + \beta SV}{VQ^\top Q + \alpha VM M^\top + \beta AV}}
\end{align*}

\begin{align*}
    \Large{M \odot M \frac{V^\top Y}{V^\top VM}}
\end{align*}

\begin{align*}
    \Large{Q \odot Q \frac{X^\top V}{QV^\top V + \gamma DQ}}
\end{align*}


In [24]:
ERS = ['Science']

dat = loadmat('../../../../Datasets/'+ ERS[0] + ".mat")
#     data = loadmat("Science.mat")
train = dat['train']
test = dat['test']

X_test = torch.from_numpy(train[0][0].T).to(torch.float32)
Y_test = torch.from_numpy(train[0][1].T).to(torch.float32)
Y_test[Y_test == -1] = 0

X = torch.from_numpy(test[0][0].T).to(torch.float32).type(dtype)
Xc = torch.from_numpy(test[0][0].T).to(torch.float32)
XTX = X.T @ X

Yc = torch.from_numpy(test[0][1].T).to(torch.float32)
Yc[Yc == -1] = 0

Y = Yc.type(dtype)
#Feature
n,d = X.shape
#label
n,c = Y.shape
#local Structure
nn = 5
sigma = 30

# #similarity & local Structure
# S = kneighbors_graph(X, nn, mode='distance', include_self=False, metric='euclidean').todense()
# S[S==0]=np.inf
# S = np.array(np.exp(-S/(sigma)))
# S = torch.from_numpy(S).to(torch.float32).type(dtype)
# #Dii
# Ds = torch.diag(torch.sum(S, dim=1))
%run LocalStructure.ipynb
S, Ds = LCS(X,sigma,nn)
alpha=0.01
beta=0.7
gamma=0.9
epsilon = torch.tensor(torch.finfo(torch.float32).eps)

#Number of selected Latent
k = 23

# coefficient Matrix
W = torch.rand(d,k).type(dtype)
V = torch.rand(n,k).type(dtype)
Q = torch.rand(d,k).type(dtype)
M = torch.rand(k,c).type(dtype)
# Number of Iteration & Parameters
iteration = 30
for o in range(iteration):
    D = torch.diag(1 / (2 * torch.linalg.norm(Q, dim=1) + epsilon))
    Va = X @ Q + alpha * (Y @ M.T) + beta * (S @ V)
    Vb = (V @ Q.T @ Q + alpha*(V @ M @ M.T) + beta * (Ds @ V))
    V = V * (Va) /torch.maximum((Vb),epsilon)
    
    Ma = V.T @ Y
    Mb = (V.T @ V @ M)
    M = M * ((Ma) / torch.maximum((Mb),epsilon))
    
    Qa = X.T @ V     
    Qb = Q @ V.T @ V + gamma * (D @ Q)
    Q = Q * ((Qa)/  torch.maximum((Qb),epsilon))
    
q = torch.linalg.norm(Q,axis=1)
sQ = torch.argsort(q)

for j in tqdm(range(20)):
    j+=1
    nosf = int (j * d / 100)
    sX = Xc[:,sQ[d-nosf:].long()]
    classifier = MLkNN(k=10)
    classifier.fit(sX.numpy(), Yc.numpy())
    #KNN
    predictions = classifier.predict(X_test[:,sQ[d-nosf:]]).toarray()
    scores = classifier.predict_proba(X_test[:,sQ[d-nosf:]]).toarray()

    Mic = f1_score(Y_test, predictions, average='micro')
    Mac = f1_score(Y_test, predictions, average='macro')
    HML = hamming_loss(Y_test,predictions)
    RNL = label_ranking_loss(Y_test,scores)
    AVP = average_precision_score(Y_test,scores)
    COV = coverage_error(Y_test,scores)


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [06:32<00:00, 19.63s/it]
